In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

import torch
import torch.nn as nn

# torch.seed(42)

# Dataset Preprocessing

Split data into train, validation and test:  
80% train, 10% validation, 10% test:

Years selected for test and validation (based of random number generation):
Valid and test years pairs are selected within one year of seperation
valid = 2022, 1999, 2008, 1990
test = 2023,  1998, 2009, 1991

Validation years

In [22]:
datafolder = "./data/"
merge_df = pd.DataFrame()
for i, file in enumerate(os.listdir(datafolder)):
    if i == 0:
        merge_df = pd.read_csv(datafolder + file)
        print(type(merge_df))
    else:
        df = pd.read_csv(datafolder + file)
        merge_df = pd.merge(merge_df, df, on="month")

merge_df["month_num"] = merge_df["month"].apply(lambda x: int(x.split("-")[1]))

merge_df


<class 'pandas.core.frame.DataFrame'>


,month,maximum_rainfall_in_a_day,no_of_rainy_days,total_rainfall,rh_extremes_minimum,mean_rh,mean_sunshine_hrs,mean_temp,temp_mean_daily_max,temp_mean_daily_min,month_num
0,1982-01,36.5,10,107.1,49,81.2,5.6,25.9,29.8,23.3,1
1,1982-02,9.4,5,27.8,49,79.5,7.6,27.1,32.3,23.9,2
2,1982-03,61.7,11,160.8,51,82.3,5.1,27.2,31.7,24.3,3
3,1982-04,45.1,14,157.0,54,85.9,4.7,27.0,31.4,24.3,4
4,1982-05,33.0,10,102.2,57,83.2,5.8,28.0,31.7,25.2,5
...,...,...,...,...,...,...,...,...,...,...,...
513,2024-10,66.0,15,147.6,41,78.9,5.0,28.5,32.1,25.8,10
514,2024-11,71.2,24,407.4,53,84.5,4.3,27.4,31.6,25.0,11
515,2024-12,76.8,23,234.4,41,80.7,4.6,27.7,31.7,25.3,12
516,2025-01,157.0,18,480.2,54,83.4,3.6,26.5,29.5,24.6,1


In [ ]:
# normalize data except for month and predicting value which is total_rainfall

# Either Gaussian Normalization or Min-Max Normalization

predicting_column = "total_rainfall"

excluded_columns = ["month", predicting_column]
# print(merge_df.columns)

#reorder the columns so that predicting value is at the end
cols = [col for col in merge_df.columns if col != predicting_column]  # Get all columns except "total_rainfall"
# print(cols)
cols.append(predicting_column)  # Add "total_rainfall" at the end
merge_df = merge_df[cols]

merge_df_normalized_gaussian = merge_df.copy()

epsilon = 1e-10

for column in merge_df_normalized_gaussian.columns:
    if column not in excluded_columns:
        merge_df_normalized_gaussian[column] = (merge_df_normalized_gaussian[column] - merge_df_normalized_gaussian[column].mean()) / merge_df_normalized_gaussian[column].std()
        


merge_df_normalized_minmax = merge_df.copy()

for column in merge_df_normalized_minmax.columns:
    if column not in excluded_columns:
        merge_df_normalized_minmax[column] = (merge_df_normalized_minmax[column] - merge_df_normalized_minmax[column].min() + epsilon) / (merge_df_normalized_minmax[column].max() - merge_df_normalized_minmax[column].min() + epsilon)
        
merge_df_normalized_minmax



Index(['month', 'maximum_rainfall_in_a_day', 'no_of_rainy_days',
       'total_rainfall', 'rh_extremes_minimum', 'mean_rh', 'mean_sunshine_hrs',
       'mean_temp', 'temp_mean_daily_max', 'temp_mean_daily_min', 'month_num'],
      dtype='object')
['month', 'maximum_rainfall_in_a_day', 'no_of_rainy_days', 'rh_extremes_minimum', 'mean_rh', 'mean_sunshine_hrs', 'mean_temp', 'temp_mean_daily_max', 'temp_mean_daily_min', 'month_num']


,month,maximum_rainfall_in_a_day,no_of_rainy_days,rh_extremes_minimum,mean_rh,mean_sunshine_hrs,mean_temp,temp_mean_daily_max,temp_mean_daily_min,month_num,total_rainfall
0,1982-01,0.168056,0.346154,0.571429,0.491979,0.439394,0.121951,0.178571,0.095238,9.090909e-12,107.1
1,1982-02,0.042593,0.153846,0.571429,0.401070,0.742424,0.414634,0.625000,0.238095,9.090909e-02,27.8
2,1982-03,0.284722,0.384615,0.642857,0.550802,0.363636,0.439024,0.517857,0.333333,1.818182e-01,160.8
3,1982-04,0.207870,0.500000,0.750000,0.743316,0.303030,0.390244,0.464286,0.333333,2.727273e-01,157.0
4,1982-05,0.151852,0.346154,0.857143,0.598930,0.469697,0.634146,0.517857,0.547619,3.636364e-01,102.2
...,...,...,...,...,...,...,...,...,...,...,...
513,2024-10,0.304630,0.538462,0.285714,0.368984,0.348485,0.756098,0.589286,0.690476,8.181818e-01,147.6
514,2024-11,0.328704,0.884615,0.714286,0.668449,0.242424,0.487805,0.500000,0.500000,9.090909e-01,407.4
515,2024-12,0.354630,0.846154,0.285714,0.465241,0.287879,0.560976,0.517857,0.571429,1.000000e+00,234.4
516,2025-01,0.725926,0.653846,0.750000,0.609626,0.136364,0.268293,0.125000,0.404762,9.090909e-12,480.2


In [ ]:
# split data

# select df

df = merge_df_normalized_minmax




# ModeL with RNN